<a href="https://colab.research.google.com/github/DenyT17/Football_Object_Detection/blob/main/Perspective_Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
%cd "/content/gdrive/MyDrive/Football_Detection"
!git clone https://github.com/FootballAnalysis/footballanalysis.git

/content/gdrive/MyDrive/Football_Detection
fatal: destination path 'footballanalysis' already exists and is not an empty directory.


In [3]:
%cd "/content/gdrive/MyDrive/Football_Detection/footballanalysis/Bird's eye view"
!pip install -r requirements.txt
!pip install ultralytics
!pip install pyflann-py3

/content/gdrive/MyDrive/Football_Detection/footballanalysis/Bird's eye view
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.1/22.1 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 67.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyflann-py3: filename=pyflann_py3-0.1.0-py3-none-any.whl size=7186987 sha256=8de1fce7e1889d579adf241fbc963ce1401af8dcf4b76e2b0cb83f690311adcc
  Stored in directory: /root/.cache/pip/wheels/d9/5f/aa/82122a3a88158eb3a45dd9d727baca078d58af3291e4a6a8ad
Successfully built pyflann-py3
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvis

In [4]:
%cd "/content/gdrive/MyDrive/Football_Detection"
!git clone https://github.com/ifzhang/ByteTrack.git
!cd ByteTrack && pip3 install -r requirements.txt
!cd ByteTrack && python3 setup.py develop
!pip install cython_bbox


import sys
sys.path.append(f"/content/gdrive/MyDrive/Football_Detection/ByteTrack")


/content/gdrive/MyDrive/Football_Detection
Cloning into 'ByteTrack'...
remote: Enumerating objects: 2007, done.
remote: Total 2007 (delta 0), reused 0 (delta 0), pack-reused 2007
Receiving objects: 100% (2007/2007), 79.60 MiB | 19.97 MiB/s, done.
Resolving deltas: 100% (1141/1141), done.
Updating files: 100% (229/229), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 39.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 24.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 53.1 MB/s eta 0:00:00
  Installin

In [8]:
!pip install onemetric --quiet
!pip install loguru
!pip install lap


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached lap-0.4.0-cp39-cp39-linux_x86_64.whl


In [26]:
%%writefile "/content/gdrive/MyDrive/Football_Detection/footballanalysis/Bird's eye view/main.py"

from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from elements.perspective_transform import Perspective_Transform
from elements.assets import transform_matrix, detect_color
from arguments import Arguments
from yolov5.utils.plots import plot_one_box
from ultralytics import YOLO

import torch
import os
import cv2
import numpy as np
import sys

from dataclasses import dataclass


    
def main(opt):

    @dataclass(frozen=True)
    class BYTETrackerArgs:
      track_thresh: float = 0.25
      track_buffer: int = 30
      match_thresh: float = 0.8
      aspect_ratio_thresh: float = 3.0
      min_box_area: float = 1.0
      mot20: bool = False

    byte_tracker = BYTETracker(BYTETrackerArgs())
    
    classNames = ['ball', 'goalkeeper', 'player', 'referee']
    colors = {
      "ball":(255,0,0),
      "goalkeeper":(255,255,0),
      "player":(0,255,255),
      "referee":(255,0,127),
    }
    # Load models
    perspective_transform = Perspective_Transform()

    model = YOLO("/content/gdrive/MyDrive/Football_Detection/runs/detect/train2/weights/best.pt")
    # Video capture
    cap = cv2.VideoCapture(r"/content/gdrive/MyDrive/Football_Detection/video/finall.mp4")
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)

    w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

    # Save output
    if opt.save:
        output_name = opt.source.split('/')[-1]
        output_name = output_name.split('.')[0] + '_output.' + output_name.split('.')[-1]

        output_path = os.path.join(os.getcwd(), 'inference/output')
        os.makedirs(output_path, exist_ok=True)
        output_name = os.path.join(os.getcwd(), 'inference/output', output_name)

        w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
        h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

        out = cv2.VideoWriter(output_name,  
                                cv2.VideoWriter_fourcc(*'mp4v'), 
                                opt.outputfps, (int(w), int(h)))


    frame_num = 0

    # Black Image (Soccer Field)
    bg_ratio = int(np.ceil(w/(3*115)))
    gt_img = cv2.imread('./inference/black.jpg')
    gt_img = cv2.resize(gt_img,(115*bg_ratio, 74*bg_ratio))
    gt_h, gt_w, _ = gt_img.shape


    while(cap.isOpened()):
        
        ret, frame = cap.read()
        bg_img = gt_img.copy()

        if ret:
            main_frame = frame.copy()
            yoloOutput = model(frame)

            # Output: Homography Matrix and Warped image 
            if frame_num % 5 ==0: # Calculate the homography matrix every 5 frames
                M, warped_image = perspective_transform.homography_matrix(main_frame)

            if yoloOutput:
                # The homography matrix is applied to the center of the lower side of the bbox.
                for r in yoloOutput:
                    boxes = r.boxes
                    for box in boxes:
                   
                      x1,y1,x2,y2 = box.xyxy[0]
                      x1,y1,x2,y2 = int(x1), int(y1), int(x2), int(y2)
                      x_center = x1+((x2 - x1)/2) 
                      y_center = y1+((y2 - y1)/2) 

                      tracks = byte_tracker.update(
                      output_results=detections2boxes(detections=player_detections),
                      img_info=frame.shape,
                      img_size=frame.shape)
                      
                      print(tracks)
                      if classNames[int(box.cls[0])] == 'referee':
                          plot_one_box([x1,y1,x2,y2], frame, colors["referee"], label="referee")

                      elif classNames[int(box.cls[0])] == 'player':
                          plot_one_box([x1,y1,x2,y2], frame, colors["player"], label="player")
                          coords = transform_matrix(M, (x_center, y_center), (h, w), (gt_h, gt_w))
                          cv2.circle(bg_img, coords, bg_ratio + 1, colors["player"], -1)

                      elif classNames[int(box.cls[0])] == 'ball':
                          coords = transform_matrix(M, (x_center, y_center), (h, w), (gt_h, gt_w))
                          cv2.circle(bg_img, coords, bg_ratio + 1, colors["ball"], -1)
                          plot_one_box([x1,y1,x2,y2], frame, colors["ball"], label="ball")
                      
                      elif classNames[int(box.cls[0])] == 'goalkeeper':
                          plot_one_box([x1,y1,x2,y2], frame, colors["goalkeeper"], label="goalkeeper")
                          coords = transform_matrix(M, (x_center, y_center), (h, w), (gt_h, gt_w))
                          cv2.circle(bg_img, coords, bg_ratio + 1,colors["goalkeeper"], -1)

            frame[frame.shape[0]-bg_img.shape[0]:, frame.shape[1]-bg_img.shape[1]:] = bg_img  
            
            if opt.view:
                cv2.imshow('frame',frame)
                if cv2.waitKey(1) & ord('q') == 0xFF:
                    break

            # Saving the output
            if opt.save:
                out.write(frame)

            frame_num += 1
        else:
            break

        sys.stdout.write(
            "\r[Input Video : %s] [%d/%d Frames Processed]"
            % (
                opt.source,
                frame_num,
                frame_count,
            )
        )

    if opt.save:
        print(f'\n\nOutput video has been saved in {output_path}!')
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':

    opt = Arguments().parse()
    with torch.no_grad():
        main(opt)


Overwriting /content/gdrive/MyDrive/Football_Detection/footballanalysis/Bird's eye view/main.py


In [27]:
%cd "/content/gdrive/MyDrive/Football_Detection/footballanalysis/Bird's eye view"
!python main.py --source "/content/gdrive/MyDrive/Football_Detection/video/finall.mp4" --save

/content/gdrive/MyDrive/Football_Detection/footballanalysis/Bird's eye view
Perspective Transform model loaded!

0: 384x640 1 goalkeeper, 18 players, 2 referees, 1127.3ms
Speed: 1.1ms preprocess, 1127.3ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)
Traceback (most recent call last):
  File "/content/gdrive/MyDrive/Football_Detection/footballanalysis/Bird's eye view/main.py", line 158, in <module>
    main(opt)
  File "/content/gdrive/MyDrive/Football_Detection/footballanalysis/Bird's eye view/main.py", line 101, in main
    output_results=detections2boxes(detections=player_detections),
NameError: name 'detections2boxes' is not defined
